In [17]:
from bs4 import BeautifulSoup
import requests
import html5lib
import re
import pandas as pd
import numpy as np

In [18]:
def clean_publish_year(raw_year):
    if type(raw_year) is None:
        return 0
    result = re.search(r"published (\b\d+)", raw_year)
    if result == None:
        return 0
    else:
        return result.group(1)

def clean_rating(raw_rating):
    if type(raw_rating) is None:
        return 0
    result = re.search(r"(\b\d+.\b\d+)", raw_rating)
    if result == None:
        return 0
    else:
        return result.group(1)

def clean_raters(raw_raters):
    if type(raw_raters) is None:
        return 0
    result = re.search(r"((?:\d+,?\d+)*) ratings", raw_raters)
    if result == None:
        return 0
    else:
        return result.group(1)

# def clean_readers(raw_readers):
#     result = re.search(r"(\b\d+)", raw_readers)
#     if result == None:
#         return "None"
#     else:
#         return result.group(1)

def solve_nonetype_error(value):
    if value is not None:
        return value.text.strip()
    else:
        return np.nan



In [19]:
books = []
for page in range(1, 25):
    response = requests.get("https://www.goodreads.com/shelf/show/horror?page=" + str(page))
    html_soup = BeautifulSoup(response.text, 'html5lib')
    book_container = html_soup.find_all('div', class_ = "elementList")
    for book in book_container:
        bookTitle = solve_nonetype_error(book.find('a', class_ = "bookTitle"))
       
        bookAuthor =solve_nonetype_error(book.find('a', class_ = "authorName"))

        # After(use comments)
        year = clean_publish_year(solve_nonetype_error(book.find('span', class_ = "greyText smallText")))
        rating = clean_rating(solve_nonetype_error(book.find('span', class_ = "greyText smallText")))
        raters = str(clean_raters(solve_nonetype_error(book.find('span', class_ = "greyText smallText")))).replace(",", "")
        int(raters)
        # # readerNum = clean_readers(book.find('span', class_ = "greyText statistic").text.strip())
        
        # Before (use comments)
        # year = solve_nonetype_error(book.find('span', class_ = "greyText smallText"))
        # rating = solve_nonetype_error(book.find('span', class_ = "greyText smallText"))
        # raters = solve_nonetype_error(book.find('span', class_ = "greyText smallText"))
        # readerNum = book.find('span', class_ = "greyText statistic").text.strip()


        book_data = [bookTitle, bookAuthor, year, rating, raters]
        books.append(book_data)
books = pd.DataFrame(books, columns=["Title", "Author", "Year", "Rating", "Raters"])
books.head()

TypeError: expected string or bytes-like object

In [16]:

books.to_csv("HorrorAfter.csv", index = False)
ata = pd.read_csv("HorrorAfter.csv")
ata.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1196 entries, 0 to 1195
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Title   1196 non-null   object 
 1   Author  1196 non-null   object 
 2   Year    1196 non-null   int64  
 3   Rating  1196 non-null   float64
 4   Raters  1196 non-null   int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 46.8+ KB
